# How to get these notebooks to work

Dataset describe here: http://archive.ics.uci.edu/ml/datasets/Online+Retail+II

wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

git clone git@github.com:franzoni/proton-Hands-on_task_Senior_Data_Scientist.git

mv online_retail_II.xlsx proton-Hands-on_task_Senior_Data_Scientist

In [ ]:
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

data_path = os.getcwd()
print('++ Directory of this notebook:')
print('\t%s'%data_path)

excel_with_path  = data_path+'/online_retail_II.xlsx'
pickle_with_path = data_path+'/online_retail_II.pkl'
csv_with_path    = data_path+'/online_retail_II.csv'

print('\n++Full path of the data file:')
print('\tdata_path =%s\n\n'%excel_with_path)

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster using it than opening excel files\n')
    df = pd.read_pickle(pickle_with_path)
else:
    print('-> picke does not exist, go to excel, and create it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)
    print('-> picke done\n')

In [ ]:
# define transaction: total cash for a single 'row' of the dataset, i.e. item_price times the number of items bought
df['Transaction'] = df.Quantity * df.Price
df=df.rename(columns={"Customer ID": "Customer_ID"})

# day, week and time are added for convenience of analysis later on
from datetime import datetime
df['InvoiceDay'] = df['InvoiceDate'].map(lambda p: p.date())
df['InvoiceWeekDay'] = df['InvoiceDate'].map(lambda p: p.weekday())
df['InvoiceTime'] = df['InvoiceDate'].map(lambda p: p.time())
df['InvoiceWeek'] = df['InvoiceDate'].map(lambda p: p.isocalendar()[1]+52*(p.year-2010))

# avoide negative weeks and start counting from the first week of the dataset, which starts from 01/12/2009
df['InvoiceWeek'] = df['InvoiceWeek']+3


# implement the definition of cancellation in the documentation:
# http://archive.ics.uci.edu/ml/datasets/Online+Retail+II
#      ==> "If this code starts with the letter 'c', it indicates a cancellation.""
def is_cancellation(s):
    """" implement the definition of cancellation in the documentation:
          http://archive.ics.uci.edu/ml/datasets/Online+Retail+II
          ==> "If this code starts with the letter 'c', it indicates a cancellation."""
    if isinstance(s, int):
        #print "ordinary invoice"
        return 0
    elif isinstance(s, unicode):
        #print "unicode string"
        #thse are cancellations
        if s.rfind('C')!=-1:
            return 1
        else:
            # print "Something unexpected"  # found, e.g. A506401
            # print s
            return 2
df['IsCancellation'] = df['Invoice'].map(is_cancellation)



max_item_price=1400
r             =(0,max_item_price)
def is_ordinary_item(p):
    """ items with prices above ~1500 are so few that is worth looking at them in detail, and excluding them from the plots
        There'a lot of transactions with price set to 0, which based on the descriptions are """
    if abs(p)>max_item_price or p==0:
        return 0
    else:
        return 1
df['IsOrdinaryItem'] = df['Price'].map(is_ordinary_item)        

# df.to_csv(csv_with_path, encoding = 'utf-8')

<a id="section_EDA_cleaning"></a>
# data cleaning: nan's

In [ ]:
# what are the nulls in each feature?
nulls = df.isnull().sum()[df.isnull().sum() != 0]

# select all transactions containing 
df_nulls = df[df.isnull().any(axis=1)].copy(deep=True)

nulls_rel = nulls/df.shape[0]*100

nulls_summary = pd.concat([nulls, nulls_rel], axis=1, keys=['nulls', 'rel. nulls [%]'])\
               .sort_values('nulls', ascending=False)

nulls_summary

## todo
- nan and empty -> cleanup   # IN THE MAKING -> only consequences on the users study
- add colums for data and time, separately ?    # DONE
- create a second DF by user: RFM added to


# What do I want to extract from this dataset? WORKING NOTES
- https://en.wikipedia.org/wiki/Exploratory_data_analysis

- how many customers   DONE, also as a function of country
- make a pliot of transaction, of purchase prices, and of invoice_price! #  DONE

- VS TIME: trends of spending overall, by country
- revenue as a function of date, as a function of the time in the day  # DONE
- trends of spending: overall, by the largest customer, by the smaller customers (TOO SPECIFIC?)

- NEED TO BUILD A PER CUSTOMER DF
- customer: how many transactions, how much total revenue:
    => BREAK DOWN BY country, tra
- RMF


- ==> do this
- how many different types of items                  # DONE
- what kind of items are bought the most (by NUMBER of by REVENUE), are cancelled the most, 
- how much revenue per type of item bought

- correlation between
- cancellations: fraction of cancellation by nunber of transactions and by proportion of renenue
-                correlation to CHURNING ? Correlation to country OR type of good purchased ?

- Can I cathegorise the purchasable items? => If so, customers split across those cathegories
- Persona; how many cheap items, few expensive ones?
- https://cxl.com/blog/creating-customer-personas-using-data-driven-research/
- now many items are bought per session, how much is spent per session

==> TOWARDS CUSTOMER PERSONA DEFINITION
- MANY TRANSACTIONS ARE RECORDED AT THE SAME TIME => as if they were a single shopping session, but billed in split goups of goods
- the function summary_data_from_transaction_data treats transactions taking place on the same day as A SINGLE ONE!

<a id="section_customers"></a>
## Customers carachteristics

# Cleanup of data for customer analysis
- we decide to remove all datata points with no Customer_ID
- and to limit the study only to IsOrdinaryItem (as defined above), to avoid skewing the resuts w/ dataa points which are not actual purchases
- to keep the modelling basic, for now we decide also to remove all cancellation oders



In [ ]:
df.shape

In [ ]:
df=df.dropna()

In [ ]:
df.shape

In [ ]:
df= df.loc[ (df.IsCancellation==0) & (df.IsOrdinaryItem==1) ]    

In [ ]:
df.shape

In [ ]:
df=df.sort_values(by=['InvoiceDate'],ascending=True)

# Basic customer integral features
- number of items bought trgought the entire period covered by data, indicating the overall commercial activity of each individual
- unique items, indicates the variety of the customers interests
- total amount spent by a customer
- number of invoices, i.e. how many times a customers surfaced and made an order

In [ ]:
print('There are %d customers in the cleaned dataset'%df['Customer_ID'].nunique() )

# Customer value, diversity of interests, number of transactions and invoices

In [ ]:
df_customer_items  = df.groupby('Customer_ID').StockCode.agg([len])     \
                         .rename(columns={"len": "items"})               \
                         .sort_values(by=['items'],ascending=False)

tit='Num items bought by customer'
r=(0,3000)
df_customer_items.plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))

In [ ]:
df_customer_uniq_items  = df.groupby('Customer_ID').StockCode.unique().agg([len])     \
                         .rename(columns={"len": "uniq_items"})               \
                         .sort_values(by=['uniq_items'],ascending=False)

tit='Num unique items bought by customer'

r=(0,750)
df_customer_uniq_items.plot(kind='hist',logy=True,title=tit, \
                            bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))
plt.xlabel('Number of items') 
df_customer_uniq_items.describe()

In [ ]:
df_customer_invoices  = df.groupby('Customer_ID').Invoice.unique().agg([len])     \
                         .rename(columns={"len": "invoices"})               \
                         .sort_values(by=['invoices'],ascending=False)

tit='Num invoices per customer'
r=(0,160)
df_customer_invoices.plot(kind='hist',logy=True, \
                          title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))
plt.xlabel('Number of invoices') 
df_customer_invoices.describe()

In [ ]:
df_customer_items['invoices_cl'] = df_customer_invoices['invoices']
df_customer_items['itemsPerInvoice'] = df_customer_items['items']/df_customer_items['invoices_cl']

df_customer_items.head()

tit='Num items per invoice'
r=(0,160)
df_customer_items.itemsPerInvoice.plot(kind='hist',logy=True, \
                          title=tit,bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))
plt.xlabel('Num items per invoice') 
df_customer_items.describe()

In [ ]:
df_customer_spent = df.groupby('Customer_ID').Transaction.agg([sum])     \
                   .rename(columns={"sum": "total_spent"})               \
                   .sort_values(by=['total_spent'],ascending=False)

tit='total amount spent by a customer'
r=(0,100000)
df_customer_spent.plot(kind='hist',logy=True,title=tit, \
                       bins=50,range=r,colormap='Accent',alpha=0.5,figsize=(7,7))
plt.xlabel('amount spent [Pounds]') 
df_customer_spent.describe()

In [ ]:
df_customer_spent.sort_values(by=['Customer_ID'],ascending=True).head()

In [ ]:
df.loc[ (df.Customer_ID==12346) &  (df.Quantity<100000)].Transaction.sum()

In [ ]:
# validation by spot checks: on a few specific customers to ensure there's no bugs
df.loc[ (df.Customer_ID==12346)]

In [ ]:
df.loc[ (df.Customer_ID==12346)].InvoiceDay.unique().shape
# verification of 'fequency' being equal to 7 for 12346
# remiminder: the RFM variables exclude the first invoice

# Historic customer  features

## The literature is rich in models and prescriptions
- see for instance: https://www.blastanalytics.com/blog/rfm-analysis-boosts-sales
- and https://joaocorreia.io/blog/rfm-analysis-increase-sales-by-segmenting-your-customers.html

## to characterise customers according to three simple historical variables 
- **Recency**: How recently a customer has made a purchase
- **Frequency**: How often a customer makes a purchase
- **Monetary Value**: Average expenditure for an invoice

# Notes
- RFM variables exlcude the first invoice for all customers. Only returning invoices are considered
- note2: values=0 are for customers that don't return => select those customers away to consider

# Preparation of data: two attempts to cluster customers
- both relying on RFM data, which I am about to set up now

In [ ]:
df.shape

In [ ]:
# pip install --user lifetimes
print('pip install worked out fine')

In [ ]:
data_star='2009-12-01'
calib_end='2011-05-31'
obser_end='2011-12-09'

from datetime import date
d_data_star=date( * map(lambda u:int(u), data_star.split('-') ) )
d_calib_end=date( * map(lambda u:int(u), calib_end.split('-') ) )
d_obser_end=date( * map(lambda u:int(u), obser_end.split('-') ) )

from lifetimes.utils import *

#    This transforms a DataFrame of transaction data of the form:
#        customer_id, datetime [, monetary_value]
#    to a DataFrame of the form:
#        customer_id, frequency, recency, T [, monetary_value]

df_customer_RFM = summary_data_from_transaction_data(df.sort_values(by=['InvoiceDate'],ascending=True)
                                                     ,'Customer_ID','InvoiceDate'
                                                     ,monetary_value_col='Transaction'
                                                     ,freq='D'
                                                     #,observation_period_end=obser_end
                                                    )

In [ ]:
df_customer_RFM.head()

In [ ]:
tit='(Monetary Value) Average expenditure for an invoice'
r=(0,3000)
df_customer_RFM['monetary_value'].loc[df_customer_RFM['monetary_value']>0]  \
                                 .plot(kind='hist',logy=True,title=tit,     \
                                       bins=50,range=r,colormap='autumn',alpha=0.5,figsize=(7,7))

plt.xlabel('Monetary Value [Pounds]') 
df_customer_RFM['monetary_value'].loc[df_customer_RFM['monetary_value']>0].describe()

In [ ]:
tit='Recency (How recently a customer has made last purchase)'
r=(0,800)
df_customer_RFM['recency'].loc[df_customer_RFM['recency']>0] \
     .plot(kind='hist',logy=True,title=tit,bins=50,          \
           range=r,colormap='autumn',alpha=0.5,figsize=(14,7))
plt.xlabel('Recency [Days]') 
df_customer_RFM['recency'].loc[df_customer_RFM['recency']>0].describe()

In [ ]:
tit='Frequency (how many times a customer has come back)'
r=(0,100)
df_customer_RFM['frequency'].loc[df_customer_RFM['frequency']>0] \
           .plot(kind='hist',logy=True,title=tit,bins=50,range=r,colormap='autumn',alpha=0.5,figsize=(7,7))
plt.xlabel('Frequency') 
df_customer_RFM['frequency'].loc[df_customer_RFM['frequency']>0].describe()

## Here's where I aggregate all specific dataframes to come up with the oveall VIEW

In [ ]:
df_customer = df_customer_items     \
                .join(df_customer_invoices)    \
                .join(df_customer_spent)   \
                .join(df_customer_RFM)   \
            .sort_values(by=['total_spent'],ascending=False)

df_customer.shape
df_customer.head()

In [ ]:
df_customer.shape

# consider only returning customers
- Keep the model simple and don''t treat customers who have never come back => require frequency and recency to be positive

In [ ]:
df_customer = df_customer.loc[df_customer['monetary_value']>0]   \
                    .loc[df_customer['recency']>0]          \
                    .loc[df_customer['frequency']>0]        \

In [ ]:
print('There are %d RETURNING customers in the cleaned dataset'%df_customer['monetary_value'].count() )
df_customer.shape

In [ ]:
df_customer.keys

In [ ]:
pd.plotting.scatter_matrix(df_customer, alpha=0.2,figsize=(15,15))

In [ ]:
# cut_out=0.02
# q_low = df_customer["monetary_value"].quantile(cut_out)
# q_hi  = df_customer["monetary_value"].quantile(1-cut_out)
# df_customer = df_customer[(df_customer["monetary_value"] < q_hi) & (df_customer["monetary_value"] > q_low)]
# df_customer.shape

In [ ]:
df_customer.corr()

# Assess the correlation between existing variables
# to makea choice of which variables to keep and discart in the clustering approach (see later on)

In [ ]:
import seaborn as sns

sns.heatmap(df_customer.drop([],axis=1).corr(),cmap="Blues")

# We keep only the 3 RFM and 'total spent'
- adding other variables would not add significan rather pollute w/ noise from highly correlated features

In [ ]:
sns.heatmap(df_customer[['frequency','monetary_value','recency','total_spent']].corr(),cmap="Blues")

## Introduce logarithm of features
- to better surface the relevance of features which are distributed exponentially
- otherwise the constraining power of most features remains hidden are otherwise hidden

In [ ]:
import numpy as np


do_log = True
if do_log:
    df_customer['monetary_value_log'] = np.log(df_customer['monetary_value'])
    df_customer['frequency_log'] = np.log(df_customer['frequency'])
    df_customer['total_spent_log'] = np.log(df_customer['total_spent'])

In [ ]:
df_customer.keys

In [ ]:
# Importing the dataset
# df_customer_km = df_customer.drop(['Customer_ID'],axis=0)

mat_customer_km = df_customer[['frequency_log','recency','monetary_value_log','total_spent_log']]
mat_customer_km.head()

In [ ]:
pd.plotting.scatter_matrix(df_customer[['frequency_log','recency','monetary_value_log','total_spent_log']], alpha=0.2,figsize=(15,15))

# APPROACH 1: basic grouping of customers
- cut-and-count method using RFM variables and their quantiles

In [ ]:
# https://www.blastanalytics.com/blog/rfm-analysis-boosts-sales
# https://joaocorreia.io/blog/rfm-analysis-increase-sales-by-segmenting-your-customers.html

In [ ]:
# functions to assign quantile bins, for the case of 4 and 3 bins

# such that 1/4 score means best/worst customer feature
def scoring_four(x,p,d,invert):
    """ the last argument in the API flips the order of the score,
        such that LARGE score always means GOOD customer feature"""
    if x <= d[p][0.25]:
        return 1 if invert else 4
    elif x <= d[p][0.50]:
        return 2 if invert else 3
    elif x <= d[p][0.75]: 
        return 3 if invert else 2
    else:
        return 4 if invert else 1

# such that 1/3 score means best/worst customer feature
def scoring_three(x,p,d,invert):
    """ the last argument in the API flips the order of the score,
        such that LARGE score always means GOOD customer feature"""
    if x <= d[p][0.33]:
        return 1 if invert else 3
    elif x <= d[p][0.66]:
        return 2 if invert else 2
    else:
        return 3 if invert else 1

In [ ]:
quantiles = df_customer.quantile(q=[0.25,0.5,0.75])
quantiles

In [ ]:
quantiles = df_customer.quantile(q=[0.33,0.66])
quantiles

In [ ]:
quantiles_d = quantiles.to_dict()
#quantiles_d 

In [ ]:
df_customer['recency_score']   = df_customer['recency']             .apply(scoring_three, args=('recency',quantiles,True))
df_customer['frequency_score'] = df_customer['frequency']           .apply(scoring_three, args=('frequency',quantiles,False))
df_customer['monetary_value_score'] = df_customer['monetary_value'] .apply(scoring_three, args=('monetary_value',quantiles,False))

In [ ]:
df_customer.head(5)

In [ ]:
df_customer.groupby(['recency_score','frequency_score','monetary_value_score'], sort = True).T.agg(len)

In [ ]:
df_customer.groupby(['monetary_value_score','recency_score',], sort = True) \
    .T.agg(len)

In [ ]:
type(df_customer.groupby(['monetary_value_score','frequency_score',], sort = True).T.agg(len))

# APPROACH 2: Customer clustering: k-mean

In [ ]:
# Using the elbow method to find the optimal number of clusters
from sklearn.cluster import KMeans
from sklearn.metrics import  silhouette_score

wcss = []
mat_customer_km = mat_customer_km.as_matrix()

import matplotlib.pyplot as plt

for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(mat_customer_km)
    clusters = kmeans.predict(mat_customer_km)
    if i>1:
        silhouette_avg = silhouette_score(mat_customer_km, clusters)
        print("For n_clusters =", i, "The average silhouette_score is :", silhouette_avg)

    wcss.append(kmeans.inertia_)
    
plt.plot(range(1, 11), wcss)
plt.title('Elbow plot')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
X=mat_customer_km
# Fitting K-Means to the dataset
how_many_clusters = 3

kmeans = KMeans(n_clusters = how_many_clusters, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)


# 0: frequency
# 1: recency
# 2: value
# 3: total
labels = ['log frequency','recency [days]','log value','log total']

fig, axs = plt.subplots(1,3)
fig.set_figheight(5)
fig.set_figwidth(16)
fig.suptitle('Clusters of customers (number of clusters = %d)'%how_many_clusters)

for x_index,y_index,u in [[0,3,0],[1,3,1],[2,3,2]]:
    
    # Visualising the clusters
    axs[u].scatter( X[y_kmeans == 1, x_index], X[y_kmeans == 1, y_index], s = 2, c = 'blue', label = 'Cluster 2')
    axs[u].scatter( X[y_kmeans == 2, x_index], X[y_kmeans == 2, y_index], s = 2, c = 'green', label = 'Cluster 3')
    axs[u].scatter( X[y_kmeans == 0, x_index], X[y_kmeans == 0, y_index], s = 2, c = 'red', label = 'Cluster 1')

    #axs[u].scatter(X[y_kmeans == 3, x_index], X[y_kmeans == 3, y_index], s = 2, c = 'cyan', label = 'Cluster 4')
    #axs[u].scatter(X[y_kmeans == 4, x_index], X[y_kmeans == 4, y_index], s = 2, c = 'magenta', label = 'Cluster 5')

    # barycenters and labels
    axs[u].scatter(kmeans.cluster_centers_[:, x_index], kmeans.cluster_centers_[:, y_index], s = 200, c = 'orange', label = 'Centroids')

    axs[u].set_xlabel(labels[x_index])
    axs[u].set_ylabel(labels[y_index])
    plt.legend()

In [ ]:
X

# interpretation and conclusions
- Customers in cluster number 2 have shopped long ago ( at least 500 days before the end of the period covered by the data) and have the largest shopping frequency and total expenditure
- Customers in cluster number 3 have shopped most recently (at most 250 days ago) and tend to have a low shopping frequency (both visible in the first plot, and the the third where total/averag expenditure is low)
- Customers in cluster number 1 have an intermediate behaviour both in terms of recency (between 250 and 500) and frequency

In [ ]:
y_kmeans 

In [ ]:
kmeans.cluster_centers_

In [ ]:
# freq
X[:,0]

In [ ]:
# recency
X[:,1]

In [ ]:
# value
X[:,2]

In [ ]:
X[0,:]

# Second Proton assignment - April 15

In [ ]:
df_customer.columns

In [ ]:
df.columns

# Cleanup of data for customer analysis
- we decide to remove all datata points with no Customer_ID
- and to limit the study only to IsOrdinaryItem (as defined above), to avoid skewing the resuts w/ dataa points which are not actual purchases
- to keep the modelling basic, for now we decide also to remove all cancellation oders

In [ ]:
# shape is ok
df.shape

In [ ]:
df.loc[ (df.Customer_ID==12346.0)  ]

In [ ]:
# df_customer_items  = df.groupby('Customer_ID').StockCode.agg([len])     \
#                          .rename(columns={"len": "items"})               \
#                         .sort_values(by=['items'],ascending=False)
df_first_time=df.groupby(['Customer_ID']).InvoiceDate.agg(min)
df_first_time.head()

In [ ]:
# df.groupby(df['A']).apply(lambda x : x.iloc[1] if len(x) >= 2 else x.iloc[0])
df_date_sorted = df.sort_values(by=['InvoiceDate'])
df_second_time = df_date_sorted.groupby(['Customer_ID']).InvoiceDate.unique().apply(lambda x : x[1] if len(x) >= 2 else '2020-04-15 11:00:00')

# df_customer_spent = df.groupby('Customer_ID').Transaction.agg([sum])     \
#                   .rename(columns={"sum": "total_spent"})               \
#                   .sort_values(by=['total_spent'],ascending=False)

df_second_time.head(100)

In [ ]:
type(df_first_time)

In [ ]:
# df_customer = df_customer_items     \
#                 .join(df_customer_invoices)    \
#                 .join(df_customer_spent)   \
#                 .join(df_customer_RFM)   \
#            .sort_values(by=['total_spent'],ascending=False)

In [ ]:
df_wfi = df.merge(df_first_time,left_on='Customer_ID', right_on='Customer_ID',suffixes=('', '_first_pur'))
# .merge(df2, left_on='lkey', right_on='rkey')

In [ ]:
df_wfi.head(2000)

In [ ]:
df_wse_bare = df_wfi.merge(df_second_time,left_on='Customer_ID', right_on='Customer_ID',suffixes=('', '_second_pur'))
df_wse_bare.head(2000)

In [ ]:
df_wse = df_wse_bare.merge(df_customer_RFM,left_on='Customer_ID', right_on='Customer_ID',suffixes=('', '_rfm'))
df_wse.head(1000)

In [ ]:
df_wse.shape

In [ ]:
df_only_first_purchases = df_wse.loc[df_wse['InvoiceDate'] == df_wse['InvoiceDate_first_pur']]

In [ ]:
df_only_first_purchases.shape

In [ ]:
df_only_second_purchases = df_wse.loc[df_wse['InvoiceDate'] == df_wse['InvoiceDate_second_pur']]

In [ ]:
df_only_second_purchases.shape

In [ ]:
pd.plotting.scatter_matrix(df_only_first_purchases[['Quantity','InvoiceDate','Price','Country','Transaction']], alpha=0.2,figsize=(15,15))

In [ ]:
maxq    = 100
maxtran = 250
pd.plotting.scatter_matrix(df_only_first_purchases[['Quantity','InvoiceDate','Price','Country','Transaction','recency']] \
                           [(df_only_first_purchases.recency<250) & (df_only_first_purchases.Quantity<maxq)  & (df_only_first_purchases.Price<150) & (df_only_first_purchases.Transaction<maxtran) ], \
                           alpha=0.2,figsize=(15,15),color='green')

In [ ]:
pd.plotting.scatter_matrix(df_only_first_purchases[['Quantity','InvoiceDate','Price','Country','Transaction','recency']] \
                           [(df_only_first_purchases.recency>=250) & (df_only_first_purchases.recency<510) & (df_only_first_purchases.Quantity<maxq)  & (df_only_first_purchases.Price<150) & (df_only_first_purchases.Transaction<maxtran)   ], \
                           alpha=0.2,figsize=(15,15),color='green')

In [ ]:
pd.plotting.scatter_matrix(df_only_first_purchases[['Quantity','InvoiceDate','Price','Country','Transaction','recency']] \
                           [(df_only_first_purchases.recency>510)  & (df_only_first_purchases.Quantity<maxq)  & (df_only_first_purchases.Price<150) & (df_only_first_purchases.Transaction<maxtran) ], \
                           alpha=0.2,figsize=(15,15),color='green')

In [ ]:
pd.plotting.scatter_matrix(df_only_second_purchases[['Quantity','InvoiceDate','Price','Country','Transaction','recency']] \
                           [(df_only_second_purchases.recency<250)   & (df_only_second_purchases.Quantity<maxq)  & (df_only_second_purchases.Price<150) & (df_only_second_purchases.Transaction<maxtran) ], \
                           alpha=0.2,figsize=(15,15),color='orange')

In [ ]:
pd.plotting.scatter_matrix(df_only_second_purchases[['Quantity','InvoiceDate','Price','Country','Transaction','recency']] \
                           [(df_only_second_purchases.recency<510) & (df_only_second_purchases.recency>=250) & (df_only_second_purchases.Quantity<maxq)  & (df_only_second_purchases.Price<150) & (df_only_second_purchases.Transaction<maxtran) ], \
                           alpha=0.2,figsize=(15,15),color='orange')

In [ ]:
pd.plotting.scatter_matrix(df_only_second_purchases[['Quantity','InvoiceDate','Price','Country','Transaction','recency']] \
                           [(df_only_second_purchases.recency>=510)& (df_only_second_purchases.Quantity<maxq)  & (df_only_second_purchases.Price<150) & (df_only_second_purchases.Transaction<maxtran)   ], \
                           alpha=0.2,figsize=(15,15),color='orange')